# 機械学習 第３回レポート課題

## 0.概要
- 配布する訓練データ（X_train.npy、y_train.npy）を用いて2クラス分類問題を学習させ、配布するテストデータ（X_test.npy）に対するyのバイナリの予測値を一次元配列の形でcsvファイルとして提出せよ。授業で学んだようなモデルたちの比較や、ハイパーパラメーターを色々サーチしたりすることでベストなマイモデルを作成し、それによる予測値を提出。ファイル名は22vr0000_JeffHinton_Jeff.csvのように、学籍番号_名前_名前2.csvとすること。「名前2」の部分はスコアと共に公開されるので、どうしても本名が嫌な場合はあだ名を自分でつけること（まぁ、結果が悪くても恥ずかしがらなくていいので、本名で構いませんよ！）。利用可能な機械学習モデルは、授業で扱ったものに限るものとする。

- 訓練用のx、y（X_train.npy、y_train.npy）と、テスト用のxデータ（X_test.npy）は、以下のフォルダにある
    - https://drive.google.com/drive/folders/1DBoFa7Re_MRRrAjdvlOKDlwpYdfHQIyR?usp=share_link

### 注意事項
1. 再現性の確認のため、作業の記録・分析をColab Notebookの形にまとめてレポートとして提出すること。
2. 手抜き・工夫のない故に性能が低い結果は評価が低いが、いろいろがんばった結果スコアが悪い場合はきちんと評価する。また簡単に多量のモデルを比較できる便利ツールを使った結果は評価しません（便利で良いツールでも、それを使っても学びはないのと思うので、この授業では評価しません）

## 1.モデル構築用スクリプト